In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
import cost_function


In [4]:
class DataTransformer():
    def __init__(self, val_size=0.25,
                 encodeTarget=False,
                 dummies=False,
                 composite_features=False,
                 scaler="standard",
                 imputer="simple",
                 only_use_compounds=False):
        
        # self.group_dictionary = pd.read_csv('data/group_dictionary.csv', sep=';')
        self.test_data_no_target = pd.read_csv('data/test_data_no_target.csv', sep=';', decimal=',')
        self.training_data = pd.read_csv('data/training_data.csv', sep=';', decimal=',')
        # self.column_names_dictionary = pd.read_csv('data/column_names_dictionary.csv', sep=';')

        if encodeTarget:
            self.labelEncodeTarget()

        if dummies:
            self.makeDummies()
        
        self.X = self.training_data.drop(columns=['Class','Perform'])  
        self.y = self.training_data[['Class']]

        
        # collect cat_cols, bin_cols, num_cols
        # num_cols are then used for scaling
        # cat_cols are gonna be encoded
        self.cat_cols = []
        self.bin_cols = []
        self.num_cols = []
        for col in self.X.columns.tolist():
            if len(self.X[col].value_counts()) > 2 and len(self.X[col].value_counts())<10:
                self.cat_cols.append(col)
            elif len(self.X[col].value_counts()) == 2:
                self.bin_cols.append(col)
            else:
                self.num_cols.append(col)

    
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(self.X, self.y, test_size=val_size, random_state=42)

        if imputer == "simple":
            self.simpleImpute(0)

        if composite_features == True:
            self.create_composite_features()
            for col in self.X.columns.tolist():
                if len(self.X[col].value_counts())>10:
                    self.num_cols.append(col)
        
        if scaler == "standard":
            self.standardScale()
        
        elif scaler == "minmax":
            self.minMaxScale()

        
    
    def get_X_train(self):
        return self.X_train

    
    def get_y_train(self):
        return self.y_train

    
    def get_X_val(self):
        return self.X_val

    
    def get_y_val(self):
        return self.y_val

    
    def makeDummies(self):
        self.training_data = pd.concat((self.training_data.drop(columns=['Group']), pd.get_dummies(self.training_data['Group'])), axis=1)
        self.test_data_no_target = pd.concat((self.test_data_no_target.drop(columns=['Group']), pd.get_dummies(self.test_data_no_target['Group'])), axis=1)
        

    def knnImpute(self):
        self.knnImputer = KNNImputer(n_neighbors=4)
        self.X_train[self.num_cols] = self.knnImputer.fit_transform(self.X_train[self.num_cols])
        self.X_val[self.num_cols] = self.knnImputer.transform(self.X_val[self.num_cols])
        self.test_data_no_target[self.num_cols] = self.knnImputer.transform(self.test_data_no_target[self.num_cols])


    def simpleImpute(self, value):
        self.simpleImputer = SimpleImputer(strategy='mean')
        self.X_train[self.num_cols] = self.simpleImputer.fit_transform(self.X_train[self.num_cols])
        self.X_val[self.num_cols] = self.simpleImputer.transform(self.X_val[self.num_cols])
        self.test_data_no_target[self.num_cols] = self.simpleImputer.transform(self.test_data_no_target[self.num_cols])


    def standardScale(self):
        self.stdScaler = StandardScaler()
        self.X_train[self.num_cols] = self.stdScaler.fit_transform(self.X_train[self.num_cols])
        self.X_val[self.num_cols] = self.stdScaler.transform(self.X_val[self.num_cols])
        self.test_data_no_target[self.num_cols] = self.stdScaler.transform(self.test_data_no_target[self.num_cols])
        

    def minMaxScale(self):
        self.minMaxScale = MinMaxScaler()
        self.X_train[self.num_cols] = self.minMaxScale.fit_transform(self.X_train[self.num_cols])
        self.X_val[self.num_cols] = self.minMaxScale.transform(self.X_val[self.num_cols])
        self.test_data_no_target[self.num_cols] = self.minMaxScale.transform(self.test_data_no_target[self.num_cols])

    
    def PCASelection(self):
        pass

    
    def labelEncodeTarget(self):
        # label encodes target to be 0, 1, 2
        self.target_encoder = LabelEncoder()
        self.training_data['Class'] = self.training_data[['Class']].apply(self.target_encoder.fit_transform)
        

    def labelDecodeTarget(self, data):
        # returns python list decoding back to -1, 0, 1
        return self.target_encoder.inverse_transform(data.ravel()).tolist()


    def getCompundFeatures(self):
        if create_composite_features:
            self.create_composite_features()
    



    def create_composite_features(self):
        # Liquidity Ratios
        self.X_train['Liquidity Ratio'] = self.X_train[['I49', 'I50', 'I52']].mean(axis=1)
        self.X_val['Liquidity Ratio'] = self.X_val[['I49', 'I50', 'I52']].mean(axis=1)
        self.test_data_no_target['Liquidity Ratio'] = self.test_data_no_target[['I49', 'I50', 'I52']].mean(axis=1)
        
        # Efficiency Ratios
        self.X_train['Efficiency Ratio'] = self.X_train[['I21', 'I22', 'I24', 'I25', 'I28']].mean(axis=1)
        self.X_val['Efficiency Ratio'] = self.X_val[['I21', 'I22', 'I24', 'I25', 'I28']].mean(axis=1)
        self.test_data_no_target['Efficiency Ratio'] = self.test_data_no_target[['I21', 'I22', 'I24', 'I25', 'I28']].mean(axis=1)
        
        # Calculate Total Equity
        self.X_train['Total Equity'] = self.X_train['I19'] / self.X_train['I53']
        self.X_val['Total Equity'] = self.X_val['I19'] / self.X_val['I53']
        self.test_data_no_target['Total Equity'] = self.test_data_no_target['I19'] / self.test_data_no_target['I53']

        # Calculate Total Assets
        self.X_train['Total Assets'] = self.X_train['Total Equity'] / (1 - self.X_train['I54'])
        self.X_val['Total Assets'] = self.X_val['Total Equity'] / (1 - self.X_val['I54'])
        self.test_data_no_target['Total Assets'] = self.test_data_no_target['Total Equity'] / (1 - self.test_data_no_target['I54'])

        # Calculate Equity Multiplier
        self.X_train['Equity Multiplier'] = self.X_train['Total Assets'] / self.X_train['Total Equity']
        self.X_val['Equity Multiplier'] = self.X_val['Total Assets'] / self.X_val['Total Equity']
        self.test_data_no_target['Equity Multiplier'] = self.test_data_no_target['Total Assets'] / self.test_data_no_target['Total Equity']

        # Leverage Ratios
        self.X_train['Leverage Ratio'] = self.X_train[['I17', 'I19', 'I55', 'I54', 'Equity Multiplier']].mean(axis=1)
        self.X_val['Leverage Ratio'] = self.X_val[['I17', 'I19', 'I55', 'I54', 'Equity Multiplier']].mean(axis=1)
        self.test_data_no_target['Leverage Ratio'] = self.test_data_no_target[['I17', 'I19', 'I55', 'I54', 'Equity Multiplier']].mean(axis=1)
        
        # Profitability Ratios
        self.X_train['Profitability Ratio'] = self.X_train[['I1', 'I2', 'I6', 'I11', 'I34', 'I35', 'I37', 'I32', 'I33', 'I38']].mean(axis=1)
        self.X_val['Profitability Ratio'] = self.X_val[['I1', 'I2', 'I6', 'I11', 'I34', 'I35', 'I37', 'I32', 'I33', 'I38']].mean(axis=1)
        self.test_data_no_target['Profitability Ratio'] = self.test_data_no_target[['I1', 'I2', 'I6', 'I11', 'I34', 'I35', 'I37', 'I32', 'I33', 'I38']].mean(axis=1)

        # Price Ratios
        self.X_train['Price Ratios'] = self.X_train[['I41', 'I42', 'I43', 'I56', 'I58', 'I44']].mean(axis=1)
        self.X_val['Price Ratios'] = self.X_val[['I41', 'I42', 'I43', 'I56', 'I58', 'I44']].mean(axis=1)
        self.test_data_no_target['Price Ratios'] = self.test_data_no_target[['I41', 'I42', 'I43', 'I56', 'I58', 'I44']].mean(axis=1)


class Modelling:
    def __init__(self):
        pass

    def randomForest():
        pass
        
    def catBoost():
        pass


    def get_leaderboard():
        pass

        
        
        

In [5]:
data = DataTransformer(encodeTarget=False, dummies=True, composite_features=False)

In [6]:
X_train = data.get_X_train()
X_val = data.get_X_val()
y_train = data.get_y_train()
y_val = data.get_y_val()


In [7]:
compounds = ["Liquidity Ratio",
             "Efficiency Ratio",
             "Total Equity",
             "Total Assets",
             "Equity Multiplier",
             "Leverage Ratio",
             "Profitability Ratio",
             "Price Ratios"
            ]

In [1]:
# neural approach



In [8]:
X_train = X_train[compounds]
X_val = X_val[compounds]


KeyError: "None of [Index(['Liquidity Ratio', 'Efficiency Ratio', 'Total Equity', 'Total Assets',\n       'Equity Multiplier', 'Leverage Ratio', 'Profitability Ratio',\n       'Price Ratios'],\n      dtype='object')] are in the [columns]"

In [17]:
y_val.values.reshape(-1,)

array([1, 1, 1, ..., 1, 0, 0])

In [19]:
from catboost import CatBoostClassifier

class CountErrors:
    '''Count number of wrong predictions'''
    
    def is_max_optimal(self):
        False # Lower is better

    def evaluate(self, approxes, target, weight):  
        y_pred = np.array(approxes).argmax(0)
        y_true = np.array(target)
        cost_matrix = np.array([[0, 1, 2], 
                                [1, 0, 1], 
                                [2, 1, 0]])

        conf_matrix = confusion_matrix(y_true, y_pred)
        loss = np.sum(conf_matrix * cost_matrix) / len(y_true)
                                    
        return loss, 1

    def get_final_error(self, error, weight):
        return error

model = CatBoostClassifier(cat_features=data.cat_cols,
                           metric_period=50,
                           n_estimators=500,
                           eval_metric=CountErrors()
                          )

model.fit(X_train, y_train, eval_set=(X_val, y_val))

Learning rate set to 0.145843
0:	learn: 0.9001667	test: 0.9315000	best: 0.9315000 (0)	total: 140ms	remaining: 1m 9s


/Users/baga_nuhkadiev/opt/anaconda3/lib/python3.8/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'confusion_matrix': Cannot determine Numba type of <class 'function'>

File "../../../var/folders/7y/57xm51hj4y11r4hg1s330bth0000gn/T/ipykernel_27290/512832910.py", line 16:
<source missing, REPL/exec in use?>

  self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_model else None)


50:	learn: 0.7698333	test: 0.9345000	best: 0.9315000 (0)	total: 385ms	remaining: 3.39s
100:	learn: 0.6798333	test: 0.9485000	best: 0.9315000 (0)	total: 580ms	remaining: 2.29s
150:	learn: 0.6206667	test: 0.9630000	best: 0.9315000 (0)	total: 805ms	remaining: 1.86s
200:	learn: 0.5691667	test: 0.9615000	best: 0.9315000 (0)	total: 1.06s	remaining: 1.58s
250:	learn: 0.5208333	test: 0.9690000	best: 0.9315000 (0)	total: 1.35s	remaining: 1.34s
300:	learn: 0.4743333	test: 0.9655000	best: 0.9315000 (0)	total: 1.54s	remaining: 1.02s
350:	learn: 0.4455000	test: 0.9675000	best: 0.9315000 (0)	total: 1.75s	remaining: 744ms
400:	learn: 0.4158333	test: 0.9725000	best: 0.9315000 (0)	total: 1.95s	remaining: 480ms
450:	learn: 0.3838333	test: 0.9610000	best: 0.9315000 (0)	total: 2.14s	remaining: 233ms
499:	learn: 0.3591667	test: 0.9530000	best: 0.9315000 (0)	total: 2.44s	remaining: 0us

bestTest = 0.9315
bestIteration = 0

Shrink model to first 1 iterations.


In [188]:
preds = model.predict(data.test_data_no_target)

In [189]:
preds

array([[ 1],
       [ 1],
       [-1],
       ...,
       [ 1],
       [-1],
       [-1]])

In [192]:
preds.reshape(2000)

array([ 1,  1, -1, ...,  1, -1, -1])

In [193]:
def submit(path, predictions):
    # writes submition file
    with open(path, 'w') as f:
        for prediction in predictions.tolist():
            f.write(f"{prediction}\n")  # Writing each prediction on a new line
    
    print("Submission file created:", path)
    return True


submit("submitions/catboost_tryout.txt", preds.reshape(2000))

Submission file created: submitions/catboost_tryout.txt


True

In [177]:
data.test_data_no_target

,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,dI49,dI50,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58
0,G3,0.039284,-0.034382,-0.040373,0.090023,-0.055953,0.049229,0.016236,-0.045865,0.692039,...,-0.377892,-0.410908,-0.417438,-0.220481,0.002526,0.124543,-0.219341,0.148324,0.020379,-0.004969
1,G9,-0.491713,-0.039484,-0.043612,-0.320312,0.065520,-0.216314,-0.366573,-0.052790,-0.370449,...,0.553799,0.660505,0.806000,0.781506,-0.012519,-0.014169,0.669341,0.015033,0.023411,0.007037
2,G7,-0.744979,-0.038271,-0.038631,-0.824021,0.117907,-1.434879,-0.707810,-0.073176,-1.698337,...,0.707252,0.909110,0.738062,-0.024403,0.032079,0.095528,0.350733,-0.149682,0.005134,1.135389
3,G2,0.694507,-0.026038,-0.038985,1.892921,-0.100102,0.022436,-0.198048,-0.050572,0.208738,...,-0.120561,-0.118927,-0.130250,-0.000468,0.002082,0.023541,-0.044464,0.033332,0.079054,-0.002624
4,G2,0.207123,-0.020462,-0.033451,-0.010191,-0.146812,0.235521,0.044640,-0.029062,0.464286,...,-0.672086,-0.679972,-1.136937,-0.789870,0.007839,-0.424394,-0.704341,0.011889,0.936284,-0.015745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,G9,-1.192924,-0.049079,-0.049806,-0.513692,-0.081669,-0.334987,-0.484038,-0.063376,-0.860113,...,0.280022,0.197899,0.425563,0.146857,0.007887,0.144111,0.556732,0.809250,0.040464,-0.000838
1996,G11,0.761006,-0.028790,-0.040586,-0.098537,0.334470,0.177993,0.733065,-0.032186,2.498567,...,-0.140446,NaN,-0.044562,0.508117,0.007100,0.492209,0.634159,-0.389522,0.030106,0.019739
1997,G4,0.663620,-0.014721,-0.035249,2.367521,0.035822,0.437514,0.241244,-0.033637,0.475282,...,-0.562151,-0.694239,-0.974187,0.052026,0.026136,0.287294,-0.278286,-0.283358,-0.005308,0.001944
1998,G1,-0.887896,-0.031149,-0.044787,NaN,3.235044,9.613638,10.032149,0.096885,0.049668,...,NaN,NaN,NaN,NaN,-0.117762,0.038035,-0.140377,-1.110764,NaN,0.000807


In [ ]:
preds = model.predict()

In [21]:
import catboost

In [24]:
# Try different loss that helps with current objective

In [23]:
import numpy as np

model = CatBoostClassifier(cat_features=data.cat_cols,
                           eval_metric=CountErrors(),
                           loss_function='MultiClass',
                           early_stopping_rounds = 100,
                           class_weights = [2, 1, 2]
                           )

PARAMS = {
    "metric_period": [50, 60],
    "max_depth": np.arange(4, 20, 1),
    # "subsample": [3, 5, 7]
    "l2_leaf_reg": np.arange(0.1, 1, 0.02),  # np.arange(0.1, 1, 0.05),
    
}
model.grid_search(PARAMS, X_train, y_train, cv=5, plot=True, refit=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

/Users/baga_nuhkadiev/opt/anaconda3/lib/python3.8/site-packages/catboost/core.py:4016: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'confusion_matrix': Cannot determine Numba type of <class 'function'>

File "../../../var/folders/7y/57xm51hj4y11r4hg1s330bth0000gn/T/ipykernel_27290/512832910.py", line 16:
<source missing, REPL/exec in use?>

  cv_result = self._object._tune_hyperparams(


0:	learn: 0.9154167	test: 0.9541667	best: 0.9541667 (0)	total: 109ms	remaining: 1m 48s
50:	learn: 0.8783333	test: 0.9258333	best: 0.9208333 (48)	total: 450ms	remaining: 8.37s
100:	learn: 0.8675000	test: 0.9258333	best: 0.9108333 (83)	total: 771ms	remaining: 6.86s
150:	learn: 0.8500000	test: 0.9358333	best: 0.9108333 (83)	total: 1.1s	remaining: 6.19s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9108333333
bestIteration = 83

0:	loss: 0.9108333	best: 0.9108333 (0)	total: 1.38s	remaining: 33m 1s
0:	learn: 0.9154167	test: 0.9541667	best: 0.9541667 (0)	total: 24.2ms	remaining: 24.2s


50:	learn: 0.8783333	test: 0.9258333	best: 0.9208333 (48)	total: 360ms	remaining: 6.7s
100:	learn: 0.8675000	test: 0.9258333	best: 0.9108333 (83)	total: 702ms	remaining: 6.24s
150:	learn: 0.8500000	test: 0.9358333	best: 0.9108333 (83)	total: 1.02s	remaining: 5.76s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9108333333
bestIteration = 83

1:	loss: 0.9108333	best: 0.9108333 (0)	total: 2.6s	remaining: 31m 12s


0:	learn: 0.9154167	test: 0.9541667	best: 0.9541667 (0)	total: 26.6ms	remaining: 26.5s
50:	learn: 0.8787500	test: 0.9258333	best: 0.9208333 (48)	total: 375ms	remaining: 6.97s
100:	learn: 0.8675000	test: 0.9258333	best: 0.9108333 (83)	total: 698ms	remaining: 6.22s
150:	learn: 0.8500000	test: 0.9358333	best: 0.9108333 (83)	total: 1.05s	remaining: 5.89s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9108333333
bestIteration = 83

2:	loss: 0.9108333	best: 0.9108333 (0)	total: 3.86s	remaining: 30m 50s
0:	learn: 0.9154167	test: 0.9541667	best: 0.9541667 (0)	total: 23.7ms	remaining: 23.7s


...

50:	learn: 0.8762500	test: 0.9308333	best: 0.9208333 (4)	total: 412ms	remaining: 7.67s
100:	learn: 0.8350000	test: 0.9191667	best: 0.9141667 (63)	total: 763ms	remaining: 6.79s
150:	learn: 0.8083333	test: 0.9108333	best: 0.9008333 (119)	total: 1.12s	remaining: 6.3s
200:	learn: 0.7829167	test: 0.9191667	best: 0.9008333 (119)	total: 1.5s	remaining: 5.96s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9008333333
bestIteration = 119

60:	loss: 0.9008333	best: 0.9008333 (54)	total: 1m 29s	remaining: 33m 45s
0:	learn: 0.9100000	test: 0.9341667	best: 0.9341667 (0)	total: 22.8ms	remaining: 22.8s


50:	learn: 0.8762500	test: 0.9308333	best: 0.9208333 (4)	total: 384ms	remaining: 7.15s
100:	learn: 0.8350000	test: 0.9191667	best: 0.9141667 (63)	total: 754ms	remaining: 6.71s
150:	learn: 0.8087500	test: 0.9091667	best: 0.9008333 (119)	total: 1.12s	remaining: 6.29s
200:	learn: 0.7833333	test: 0.9175000	best: 0.9008333 (119)	total: 1.49s	remaining: 5.9s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9008333333
bestIteration = 119

61:	loss: 0.9008333	best: 0.9008333 (54)	total: 1m 31s	remaining: 33m 47s
0:	learn: 0.9100000	test: 0.9341667	best: 0.9341667 (0)	total: 22.6ms	remaining: 22.6s


50:	learn: 0.8762500	test: 0.9308333	best: 0.9208333 (4)	total: 418ms	remaining: 7.77s
100:	learn: 0.8350000	test: 0.9191667	best: 0.9141667 (63)	total: 778ms	remaining: 6.92s
150:	learn: 0.8087500	test: 0.9091667	best: 0.9008333 (118)	total: 1.16s	remaining: 6.53s
200:	learn: 0.7845833	test: 0.9175000	best: 0.9008333 (118)	total: 1.56s	remaining: 6.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9008333333
bestIteration = 118

62:	loss: 0.9008333	best: 0.9008333 (54)	total: 1m 32s	remaining: 33m 50s
0:	learn: 0.9100000	test: 0.9341667	best: 0.9341667 (0)	total: 22.7ms	remaining: 22.7s


50:	learn: 0.8762500	test: 0.9308333	best: 0.9208333 (4)	total: 374ms	remaining: 6.96s
100:	learn: 0.8350000	test: 0.9191667	best: 0.9141667 (63)	total: 753ms	remaining: 6.7s
150:	learn: 0.8091667	test: 0.9091667	best: 0.9008333 (118)	total: 1.15s	remaining: 6.47s
200:	learn: 0.7858333	test: 0.9175000	best: 0.9008333 (118)	total: 1.52s	remaining: 6.05s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9008333333
bestIteration = 118

63:	loss: 0.9008333	best: 0.9008333 (54)	total: 1m 34s	remaining: 33m 53s
0:	learn: 0.9100000	test: 0.9341667	best: 0.9341667 (0)	total: 22.6ms	remaining: 22.6s


CatBoostError: catboost/python-package/catboost/helpers.cpp:44: Traceback (most recent call last):
  File "_catboost.pyx", line 1332, in _catboost._MetricEval
  File "_catboost.pyx", line 1551, in _catboost._try_jit_method.new_method
  File "/var/folders/7y/57xm51hj4y11r4hg1s330bth0000gn/T/ipykernel_27290/512832910.py", line 16, in evaluate
    conf_matrix = confusion_matrix(y_true, y_pred)
KeyboardInterrupt


In [163]:
from sklearn.utils.class_weight import compute_class_weight
 
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train.values), y=y_train.values)
# class_weights = dict(zip(classes, weights))

TypeError: unhashable type: 'numpy.ndarray'

In [161]:
y_train

,Class
3836,1
6408,1
4840,1
527,1
6105,1
...,...
5226,1
5390,-1
860,-1
7603,-1


In [168]:
baseline = print(cost_function.evaluate_error([0] * len(y_val),y_val))

0.8655


,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
0,-0.039702,-0.110934,-0.093235,0.089020,-0.048738,0.023660,-0.006359,-0.092094,0.727864,-0.105462,...,0,0,0,1,0,0,0,0,0,0
1,-0.626151,-0.138503,-0.109433,-0.351719,0.072964,-0.250281,-0.393670,-0.123525,-0.311781,-0.125845,...,0,0,0,0,0,0,0,0,0,1
2,-0.905865,-0.131948,-0.084526,-0.892751,0.125449,-1.507389,-0.738919,-0.216046,-1.611120,-0.143551,...,0,0,0,0,0,0,0,1,0,0
3,0.683946,-0.065844,-0.086297,2.025506,-0.092970,-0.003980,-0.223163,-0.113458,0.254954,-0.038914,...,0,0,1,0,0,0,0,0,0,0
4,0.145665,-0.035710,-0.058622,-0.018620,-0.139768,0.215845,0.022379,-0.015832,0.505007,-0.041426,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-1.400589,-0.190351,-0.140412,-0.559429,-0.074503,-0.372709,-0.512515,-0.171568,-0.790917,-0.179268,...,0,0,0,0,0,0,0,0,0,1
1996,0.757389,-0.080713,-0.094304,-0.113512,0.342420,0.156497,0.718898,-0.030013,2.495552,-0.013698,...,0,1,0,0,0,0,0,0,0,0
1997,0.649833,-0.004686,-0.067609,2.535272,0.043210,0.424227,0.221294,-0.036597,0.515767,-0.026384,...,0,0,0,0,1,0,0,0,0,0
1998,-1.063707,-0.093459,-0.115311,0.000000,3.248457,9.890585,10.127324,0.555772,0.099303,-0.070779,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# try SVM

from sklearn.svm import SVC

svc = SVC()

svc.fit(X_train, y_train['Class'].ravel())
preds = svc.predict(X_val)

In [30]:
cost_function.evaluate_error(preds, y_val)

0.9215

In [ ]:
# try 